# 模型的保存

当我们训练完一个模型之后，我们可以将其保存起来，留着下次用～

这里我们以 LeNet 为例

我们有一个训练好的 LeNet 我们需要保存，有两种方法：

1、保存结构和参数

```
torch.save(model, PATH) # 保存模型的结构和参数

model = torch.load(PATH) # 加载模型的结构和参数

# PATH 为模型保存路径

```

2、仅保存参数

``` 
torch.save(model.state_dict(), PATH) # 仅仅保存模型的参数

model = ModelClass(*args, **kwargs) # 实例化模型
model.load_state_dict(torch.load(PATH)) # 加载模型参数

# PATH 为模型保存路径
```

解释一下： 对于pytorch而言，它内部每一层的参数，都会被保存在 model.state_dict() 字典中，这是一个有序字典，字典的 key 就是层的名字+ weight 或者 bias

比如：第一层就是 

> 'conv1.weight', 'conv1.bias'

然后，我们在保存模型的时候，直接把这个字典保存起来。

等到加载的时候，一层一层的遍历，把新模型的 conv1 层用我们保存好的模型参数进行替换。

这就是为什么，我们仅仅保存权重的话，可以随意的对模型的结构进行更改，但是请注意**我们必须保证模型中各层的名字一致，不一致会报错**


**推荐使用第二种保存方式，因为我们只保存参数的话，首先我们可以节省存储空间，其次加载速度也会更快，最后，我们可以对重新构建的模型进行微调，比如新增几层，或者删除几层**


In [9]:
import torch
import torch.nn as nn

class LeNet5(nn.Module):
    def __init__(self, in_dim, n_class):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, n_class)
        
        # 参数初始化函数
        for p in self.modules():
            if isinstance(p, nn.Conv2d):
                nn.init.xavier_normal(p.weight.data)
            elif isinstance(p, nn.Linear):
                nn.init.normal(p.weight.data)

    # 向前传播
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


## 第一种：保存模型的参数和结构

In [11]:
# 实例化模型
lenet = LeNet5(224, 10)

# 让我们假设，经过了一连串的训练
# 这时候的模型已经被我们训练的
# 十分完美了。

PATH = "./test.pkl"

torch.save(lenet, PATH)

/usr/local/lib/python3.5/dist-packages/torch/serialization.py:159: UserWarning: Couldn't retrieve source code for container of type LeNet5. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [13]:
model = torch.load(PATH)

LeNet5(
  (conv1): Conv2d(224, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## 第二种：仅保存模型的参数

In [15]:
torch.save(lenet.state_dict(), PATH)

In [16]:
model2 = LeNet5(224, 10) # 实例化模型
model2.load_state_dict(torch.load(PATH)) # 加载模型参数

In [17]:
print(model2)

LeNet5(
  (conv1): Conv2d(224, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
